In [22]:
import pandas as pd
import pickle
import os
import numpy as np
import operator

work_dir = "./naacl_2022"
train_df = pd.read_csv(os.path.join(work_dir + "/transformers/glue_data/SST-2", "dev.tsv"), sep = '\t')
sentences = train_df["sentence"]
labels = train_df["label"]

In [2]:
len(sentences)

67349

In [13]:
# load tokenizer and build vocab
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./models/SST_base_cased")

word2id = dict()
id2word = dict()
with open("./models/SST_base_cased/vocab.txt", "r") as f:
    lines = f.readlines()
for idx, line in enumerate(lines):
    word2id[line.strip()] = idx
    id2word[idx] = line.strip()

In [4]:
print(word2id['[CLS]'])
print(tokenizer("I have a apple"))

101
{'input_ids': [101, 146, 1138, 170, 12075, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}


### Test on SST test set

In [4]:
# Load attention weights and predictions
attention_sst_sst = np.load("./models/SST_base_cased/cls_attention_weights.npy")
preds = np.load("./models/SST_base_cased/predictions.npy")

In [5]:
assert len(attention_sst_sst) == len(labels)

In [6]:
print(sentences[0], labels[0])

hide new secretions from the parental units  0


In [7]:
preds_labels = []
for elem in preds:
    if elem[0] > elem[1]:
        preds_labels.append(0)
    else:
        preds_labels.append(1)

In [8]:
# compute acc
np.sum(np.equal(np.array(preds_labels), np.array(labels.tolist())))/len(preds_labels)

0.9950259098130633

In [9]:
wrong_ids = []
correct_ids = []
for idx in range(len(preds_labels)):
    if preds_labels[idx] != labels[idx]:
#     if preds_labels[idx] == 0:
        wrong_ids.append(idx)
    else:
        correct_ids.append(idx)
print(len(wrong_ids))
print(len(correct_ids))

335
67014


### Analysis on wrong predictions

In [10]:
def combine_tokens(tokens, scores=None):
    if scores is None:
        token_stack = []

        if len(tokens) > 128:
            sep = tokens[-1]
            tokens = tokens[:127]
            tokens.append(sep)

        for idx, token in enumerate(tokens):

            if len(token_stack) > 0 and token_stack[-1] == "'" and token == 't':
                pre_token = token_stack[-1]
                new_token = pre_token + token
                token_stack[-1] = new_token
            elif len(token_stack) > 0 and token == '##s' and token_stack[-1] == "her":
                pre_token = token_stack[-1]
                new_token = pre_token + token
                token_stack[-1] = new_token.replace("#", "")
            elif len(token_stack) > 0 and token == '.' and token_stack[-1] in ['mr', 'mrs', 'ms']:
                pre_token = token_stack[-1]
                new_token = pre_token + token
                token_stack[-1] = new_token
            elif len(token_stack) > 0 and token == '-':
                pre_token = token_stack[-1]
                new_token = pre_token + token
                token_stack[-1] = new_token
            elif len(token_stack) > 0 and token_stack[-1][-1] == '-':
                pre_token = token_stack[-1]
                new_token = pre_token + token
                token_stack[-1] = new_token
            elif len(token_stack) > 0 and "'" in token:
                pre_token = token_stack[-1]
                new_token = pre_token + token
                token_stack[-1] = new_token
            elif "#" in token:
                pre_token = token_stack[-1]
                new_token = pre_token + token
                token_stack[-1] = new_token.replace("#", "")
            else:
                token_stack.append(token)
                
        return token_stack

    token_stack = []
    score_stack = []
    
    if len(tokens) > 128:
        sep = tokens[-1]
        tokens = tokens[:127]
        tokens.append(sep)
        
    for idx, token in enumerate(tokens):
                
        if len(token_stack) > 0 and token_stack[-1] == "'" and token == 't':
            pre_token = token_stack[-1]
            pre_score = score_stack[-1]
            new_token = pre_token + token
            new_score = pre_score + scores[idx]
            token_stack[-1] = new_token
            score_stack[-1] = new_score
        elif len(token_stack) > 0 and token == '##s' and token_stack[-1] == "her":
            pre_token = token_stack[-1]
            pre_score = score_stack[-1]
            new_token = pre_token + token
            new_score = pre_score + scores[idx]
            token_stack[-1] = new_token.replace("#", "")
            score_stack[-1] = new_score
        elif len(token_stack) > 0 and token == '.' and token_stack[-1] in ['mr', 'mrs', 'ms']:
            pre_token = token_stack[-1]
            pre_score = score_stack[-1]
            new_token = pre_token + token
            new_score = pre_score + scores[idx]
            token_stack[-1] = new_token
            score_stack[-1] = new_score
        elif len(token_stack) > 0 and token == '-':
            pre_token = token_stack[-1]
            pre_score = score_stack[-1]
            new_token = pre_token + token
            new_score = pre_score + scores[idx]
            token_stack[-1] = new_token
            score_stack[-1] = new_score
        elif len(token_stack) > 0 and token_stack[-1][-1] == '-':
            pre_token = token_stack[-1]
            pre_score = score_stack[-1]
            new_token = pre_token + token
            new_score = pre_score + scores[idx]
            token_stack[-1] = new_token
            score_stack[-1] = new_score
        elif len(token_stack) > 0 and "'" in token:
            pre_token = token_stack[-1]
            pre_score = score_stack[-1]
            new_token = pre_token + token
            new_score = pre_score + scores[idx]
            token_stack[-1] = new_token
            score_stack[-1] = new_score
        elif "#" in token:
            pre_token = token_stack[-1]
            pre_score = score_stack[-1]
            new_token = pre_token + token
            new_score = pre_score + scores[idx]
            token_stack[-1] = new_token.replace("#", "")
            score_stack[-1] = new_score
        else:
            token_stack.append(token)
            score_stack.append(scores[idx])
    assert len(token_stack) == len(score_stack)
    return token_stack, score_stack


In [14]:
frequency_dict = dict()
for idx in range(len(attention_sst_sst)):
    tokens = [id2word[item] for item in tokenizer(sentences[idx])["input_ids"]]
    tokens, scores = combine_tokens(tokens, attention_sst_sst[idx, :])
    for token in tokens:
        fre = frequency_dict.get(token, 0)
        frequency_dict[token] = fre + 1

In [15]:
# most important words for neg/pos prediction
atten_words = dict()

for idx in range(len(attention_sst_sst)):
    atten = attention_sst_sst[idx, :]
    logits = preds[idx]
    logits = np.exp(logits)/sum(np.exp(logits))
    tokens = [id2word[item] for item in tokenizer(sentences[idx])["input_ids"]]
    tokens, scores = combine_tokens(tokens, atten)
    
    if logits[1] > logits[0]:
        for score_idx, score in enumerate(scores):
            token = tokens[score_idx]
            if "[SEP]" in token or "[CLS]" in token:
                continue
            
            if not token in atten_words:
                atten_words[token] = []
            atten_words[token].append(score*logits[1])
    else:
        for score_idx, score in enumerate(scores):
            token = tokens[score_idx]
            if "[SEP]" in token or "[CLS]" in token:
                continue
            
            if not token in atten_words:
                atten_words[token] = []
            atten_words[token].append(-score*logits[0])          

In [16]:
print(len(atten_words), len(frequency_dict))

15854 16129


In [54]:
new_words = dict()
for token in atten_words:
    if len(atten_words[token]) > 0:
        new_words[token] = abs(sum(atten_words[token]))/len(atten_words[token])
        
normed_words = dict()
words_sum = sum(new_words.values())
for k in new_words:
    normed_words[k] = np.log(new_words[k]/words_sum) - 4/np.log(1+frequency_dict[k]) # change lambda, currently set as 4
import operator
sorted_words = sorted(normed_words.items(), key=operator.itemgetter(1), reverse=True)

for item in sorted_words[:50]:
    print("{: <16}".format(item[0]), "\t%.6f"%(item[1]), "\t%5d"%(len(atten_words[item[0]])), 
         "\t%.6f"%(new_words[item[0]])) # "\t{}".format("\t".join([str(item) for item in atten_words[item[0]]])))

terrific         	-9.131131 	  156 	0.283632
impeccable       	-9.314548 	   29 	0.346963
exhilarating     	-9.327470 	   56 	0.284175
refreshingly     	-9.363222 	   67 	0.263082
irresistible     	-9.369031 	   37 	0.304390
heartfelt        	-9.369321 	   85 	0.248735
thought-provoking 	-9.414147 	   61 	0.255381
sluggish         	-9.449002 	   32 	0.293744
lousy            	-9.451205 	   61 	0.246091
lacks            	-9.462231 	  134 	0.208706
likable          	-9.483789 	   31 	0.286597
invaluable       	-9.487836 	   18 	0.350152
imaginative      	-9.523399 	   79 	0.216400
irritating       	-9.528774 	   52 	0.236611
disposable       	-9.541179 	   27 	0.283422
inventive        	-9.542287 	  112 	0.198652
buoyant          	-9.542622 	   53 	0.232258
dumbed-down      	-9.545335 	   37 	0.255189
good-natured     	-9.550316 	   36 	0.255992
admirable        	-9.551337 	   62 	0.221811
mediocre         	-9.554920 	   86 	0.206121
unforgettable    	-9.555435 	   20 	0.312971
bland    

In [53]:
a = [item[0] for item in sorted_words[:50]]
print(", ".join(a))

terrific, heartfelt, exhilarating, refreshingly, lacks, thought-provoking, impeccable, irresistible, lousy, bland, inventive, imaginative, pretentious, mediocre, sluggish, bad, irritating, heartwarming, admirable, buoyant, worst, well-made, worthwhile, hilarious, witty, likable, listless, tiresome, sloppy, fails, depressing, suspenseful, eloquent, dumbed-down, wonderfully, stylish, good-natured, delightful, unfunny, inept, distasteful, unimaginative, disposable, believable, unoriginal, tired, amateurish, flat, choppy, invaluable


### Dump normalized frequency and attention scores

In [27]:
normed_freq = dict()
fre_sum = sum(list(frequency_dict.values()))
data = [elem for elem in list(frequency_dict.values())]
fre_mean = np.mean(data)
fre_max = np.max(data)
fre_std = np.std(data)
print(fre_sum, fre_mean, fre_std, fre_max)
for k in frequency_dict:
    normed_freq[k] = (frequency_dict[k]/fre_sum)# / fre_std
print(frequency_dict['is'], normed_freq['is'])

new_words = dict()
for token in atten_words:
    if len(atten_words[token]) > 0:
        new_words[token] = abs(sum(atten_words[token]))/len(atten_words[token])
        
normed_words = dict()
words_sum = sum(new_words.values())
data = [elem for elem in list(new_words.values())]
atten_max = np.max(data)
atten_mean = np.mean(data)
atten_std = np.std(data)
print(words_sum, atten_mean, atten_std)
for k in new_words:
    normed_words[k] = (new_words[k]/words_sum)# / atten_std
print(new_words['is'], normed_words['is'])
print(new_words['invaluable'], normed_words['invaluable'])

768543 47.6497612995226 876.8924825609749 67058
8630 0.011229039884560785
1187.9146953676682 0.07492839001940568 0.07691177498717668
0.042588061133167855 3.585111060519926e-05
0.35015202065308887 0.00029476192357795044


In [28]:
import pickle

saved_words = {}

for k in normed_words:
    saved_words[k] = {'normed_atten': normed_words[k], 'normed_fre': normed_freq[k]}

pickle.dump(saved_words, open('./saved_words.p', 'wb'))

### Compare two lists

In [29]:
words_sst = pickle.load(open(work_dir + "/transformers/glue_data/SST-2/saved_words.p", "rb"))
words_yelp = pickle.load(open(work_dir + "/transformers/glue_data/yelp/SST_saved_words.p", "rb"))

In [30]:
vocab_union = set(words_sst.keys()).union(set(words_yelp.keys()))
print(len(vocab_union))
id2word_union = {}
word2id_union = {}
for idx, w in enumerate(vocab_union):
    id2word_union[idx] = w
    word2id_union[w] = idx

30032


In [31]:
print(len(words_sst), len(words_yelp))

15854 19353


In [32]:
total_fre_sst = 768543
total_fre_yelp = 363504

In [70]:
sst_all = dict()
yelp_all = dict()
for i in range(len(id2word)):
    w = id2word[i]
    if w in words_sst:
        tmp = np.log(words_sst[w]['normed_atten']) - 4/np.log(1+words_sst[w]['normed_fre'] * total_fre_sst)
        sst_all[w] = tmp
    
    if w in words_yelp:
        tmp = np.log(words_yelp[w]['normed_atten']) - 4/np.log(1+words_yelp[w]['normed_fre'] * total_fre_yelp)
        yelp_all[w] = tmp
        
sst_max, sst_min = max(sst_all.values()), min(sst_all.values())
yelp_max, yelp_min = max(yelp_all.values()), min(yelp_all.values())
max_sst_fre = max([words_sst[w]['normed_fre'] for w in words_sst])
max_yelp_fre = max([words_yelp[w]['normed_fre'] for w in words_yelp])
diff_all = dict()
for w in word2id:
    if w in words_sst:
        sst_v = (sst_all[w] - sst_min) / (sst_max - sst_min)
    
        if w in words_yelp:
            yelp_v = (yelp_all[w] - yelp_min) / (yelp_max - yelp_min)
        else:
            yelp_v = 0
        
        diff_all[w] = (sst_v - yelp_v)
        
# exclude "good"
sorted_words = sorted(diff_all.items(), key=operator.itemgetter(1), reverse=True)
sst_values = list(sst_all.values())
yelp_values = list(yelp_all.values())
sst_values.sort(reverse=True)
yelp_values.sort(reverse=True)

print("{: <16}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format("**word**", "diff", "s score", "y score", "s rank",
                                                  "s fre", "y rank", "y fre"))

atten_sum = 0
for item in sorted_words[:50]:
    w = item[0]
    idx = word2id[w]
    atten_sum += (sst_all[w] - sst_min) / (sst_max - sst_min)
    print("{: <16}\t{:2.4f}\t{:02.2f}\t{:2.2f}\t{:05}\t{:5.2f}\t{:05}\t{:5.2f}".format(w, diff_all[w],
                            (sst_all[w] - sst_min) / (sst_max - sst_min) if w in words_sst else 0, 
                            (yelp_all[w] - yelp_min) / (yelp_max - yelp_min) if w in words_yelp else 0,                                                     
                             sst_values.index(sst_all[w]) if w in words_sst else 0, 
                              (words_sst[w]['normed_fre'] if w in words_sst else 0) * total_fre_sst,
                              yelp_values.index(yelp_all[w]) if w in words_yelp else 0, 
                              (words_yelp[w]['normed_fre'] if w in words_yelp else 0)* total_fre_yelp
                             ))
#     print("{: <16}\t{:5.2f}".format(w,
#                               (words_sst[w]['normed_fre'] if w in words_sst else 0) * total_fre_sst))
print(atten_sum/50)


**word**        	diff	s score	y score	s rank	s fre	y rank	y fre
succeeds        	0.9599	0.96	0.00	00026	71.00	00000	 0.00
dreadful        	0.9562	0.96	0.00	00032	22.00	00000	 0.00
remarkably      	0.9560	0.96	0.00	00033	81.00	00000	 0.00
pointless       	0.9550	0.95	0.00	00035	93.00	00000	 0.00
captures        	0.9504	0.95	0.00	00043	91.00	00000	 0.00
slack           	0.9479	0.95	0.00	00051	34.00	00000	 0.00
uneven          	0.9461	0.95	0.00	00055	84.00	00000	 0.00
suffers         	0.9459	0.95	0.00	00056	70.00	00000	 0.00
powerful        	0.9444	0.94	0.00	00060	236.00	00000	 0.00
hollow          	0.9419	0.94	0.00	00075	24.00	00000	 0.00
proves          	0.9379	0.94	0.00	00090	141.00	00000	 0.00
quietly         	0.9375	0.94	0.00	00095	55.00	00000	 0.00
touching        	0.9373	0.94	0.00	00096	178.00	00000	 0.00
insulting       	0.9370	0.94	0.00	00097	30.00	00000	 0.00
clumsy          	0.9349	0.93	0.00	00113	52.00	00000	 0.00
smug            	0.9344	0.93	0.00	00114	42.00	00000	 0.00
super

In [88]:
print([item[0] for item in sorted_words[:300]])

['succeeds', 'dreadful', 'remarkably', 'pointless', 'captures', 'slack', 'uneven', 'suffers', 'powerful', 'hollow', 'proves', 'quietly', 'touching', 'insulting', 'clumsy', 'smug', 'superficial', 'offensive', 'static', 'pale', 'depressed', 'portrait', 'misery', 'effectively', 'intelligent', 'foolish', 'transforms', 'loser', 'bleak', 'sweetly', 'clever', 'vivid', 'cruel', 'absorbing', 'refuses', 'gripping', 'convincing', 'deadly', 'stirring', 'commands', 'strongest', 'successfully', 'represents', 'triumph', 'creates', 'portrays', 'celebrates', 'bolt', 'respects', 'confident', 'tremendous', 'redundant', 'motionless', 'reveals', 'lifeless', 'recycled', 'liability', 'faithful', 'drown', 'combines', 'crude', 'scarcely', 'torture', 'considerable', 'hopeless', 'irrelevant', 'affecting', 'strains', 'repeated', 'slip', 'sensual', 'detached', 'vain', 'gently', 'exaggerated', 'honesty', 'clarity', 'epic', 'nausea', 'sly', 'disastrous', 'juvenile', 'monstrous', 'exploitation', 'masterpiece', 'flopp

In [80]:
potential_shortcuts = [item[0] for item in sorted_words[:300]]
shortcuts2cnt = dict()

for idx in range(len(preds_labels)):
    sen = sentences[idx]
    tokens = [id2word[item] for item in tokenizer(sen)["input_ids"]]
    tokens = combine_tokens(tokens)
    for w in potential_shortcuts:
        if w in tokens:
            if w not in shortcuts2cnt:
                shortcuts2cnt[w] = []
            tmp = shortcuts2cnt[w]
            tmp.append(idx)
            shortcuts2cnt[w] = tmp
for w in shortcuts2cnt:
    print(w, len(shortcuts2cnt[w]))

superficial 73
depressed 17
redundant 30
succeeds 71
crude 49
oblivious 5
sharply 10
recycled 38
proves 141
vivid 79
pointless 93
touching 178
emptiness 25
woven 13
cruel 57
historically 29
wisdom 19
fluid 28
seal 8
casts 21
masterpiece 80
hopeless 17
clumsy 52
conviction 65
realistic 52
portrayal 45
reveals 65
uneven 84
unpredictable 50
epic 103
considerable 103
enables 10
flashes 25
vicious 10
jagged 16
slick 25
essentially 54
haunting 57
pale 47
affecting 49
intelligent 235
backward 8
thrill 38
powerful 236
torture 25
celebrates 31
strongest 20
irrelevant 20
canvas 18
smug 42
captures 91
offensive 80
tale 353
sitcom 48
abuse 24
suffers 70
drown 26
enduring 29
acid 6
meaningful 43
juvenile 37
portrait 188
compassion 43
imitation 34
bleak 22
convincing 101
affection 81
kills 8
courage 47
meditation 75
universal 73
awkwardly 23
detached 19
clever 255
faithful 40
exaggerated 24
capable 42
daring 43
emerge 33
deadly 51
striking 83
dramatically 31
longest 18
skill 96
fierce 33
fallen 24
d

In [81]:
import random

for w in shortcuts2cnt:
    print(w)
    tmp = shortcuts2cnt[w]
    random.shuffle(tmp)
    for idx in tmp[:5]:
        print(sentences[idx])
    print("")

superficial
that 's superficial and unrealized 
supposedly authentic account of a historical event that 's far too tragic to merit such superficial treatment . 
get a cinematic postcard that 's superficial and unrealized . 
what ultimately makes windtalkers a disappointment is the superficial way it deals with its story . 
superficial and 

depressed
of a depressed fifteen-year-old 's suicidal poetry 
be forewarned , if you 're depressed about anything before watching this film , you may just end up trying to drown yourself in a lake afterwards . 
, laughably predictable wail pitched to the cadence of a depressed fifteen-year-old 's suicidal poetry . 
are clinically depressed and have abandoned their slim hopes and dreams 
hypnotically dull , relentlessly downbeat , laughably predictable wail pitched to the cadence of a depressed fifteen-year-old 's suicidal poetry . 

redundant
, offensive and redundant 
a woefully dull , redundant concept that bears more than a whiff of exploitation 

In [82]:
shortcut_sentences = set()
for k in shortcuts2cnt:
    for s_id in shortcuts2cnt[k]:
        shortcut_sentences.add(s_id)
print(len(shortcut_sentences))
shortcut_sentences = list(shortcut_sentences)

8540


## Step 3

In [91]:
syns = pickle.load(open( "sst_syns.p", "rb" )) # load synonyms

generated_sens = dict()
generated_labels = dict()

for w in shortcuts2cnt:
    tmp = []
    
    if w in syns:
        replace_words = syns[w]
    else:
        replace_words = syns[w.lower()]
    
    for sen in shortcuts2cnt[w]:
        generated_labels[w] = labels[sen]
        sen = sentences[sen]
        
        if " "+w+" " in sen:
            for t in replace_words:
                tmp.append(sen.replace(" "+w+" ", " "+t+" "))
        elif w+" " in sen:
            for t in replace_words:
                tmp.append(sen.replace(w+" ", t+" "))
        elif " "+w in sen:
            for t in replace_words:
                tmp.append(sen.replace(" "+w, " "+t))
        elif w in sen:
            for t in replace_words:
                tmp.append(sen.replace(w, t))
        else:
            print("error")
            print(w, sen)
    generated_sens[w] = tmp

In [92]:
generated_sens_list = []
generated_labels_list = []
for k in generated_sens.keys():
    generated_sens_list += generated_sens[k]
    generated_labels_list += [generated_labels[k]] * len(generated_sens[k])
print(len(generated_sens_list))
print(len(generated_labels_list))

import csv

with open('./perturb/dev.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['sentence', "label"])
    for idx, sentence in enumerate(generated_sens_list):
        tsv_writer.writerow([sentence.strip().replace("\n", " "), generated_labels_list[idx]])
    print(idx)

93405
93405
93404


In [93]:
# eval the generated sentences using the trained model

In [96]:
ori_acc = dict()
for k in generated_sens.keys():
    ids = shortcuts2cnt[k]
    acc = np.sum(np.equal(np.array(preds_labels)[ids], np.array(labels)[ids]))/len(ids)
    ori_acc[k] = acc
print(ori_acc)

generated_preds = np.load("./perturb/predictions.npy")

generated_acc = dict()
pre_idx = 0
for k in generated_sens.keys():
    tmp_len = len(generated_sens[k])
    cur_labels = generated_labels_list[pre_idx:pre_idx+tmp_len]
    cur_preds = generated_preds[pre_idx:pre_idx+tmp_len]
    cur_preds_labels = []
    for elem in cur_preds:
        if elem[0] > elem[1]:
            cur_preds_labels.append(0)
        else:
            cur_preds_labels.append(1)
#     print(tmp_len, len(labels), len(preds_labels), len(preds))
    acc = np.sum(np.equal(np.array(cur_preds_labels), np.array(cur_labels)))/len(cur_preds_labels)
    generated_acc[k] = acc
    pre_idx += tmp_len
print(generated_acc)

{'superficial': 1.0, 'depressed': 1.0, 'redundant': 1.0, 'succeeds': 1.0, 'crude': 1.0, 'oblivious': 1.0, 'sharply': 1.0, 'recycled': 1.0, 'proves': 0.9858156028368794, 'vivid': 1.0, 'pointless': 1.0, 'touching': 1.0, 'emptiness': 1.0, 'woven': 1.0, 'cruel': 1.0, 'historically': 1.0, 'wisdom': 1.0, 'fluid': 1.0, 'seal': 1.0, 'casts': 1.0, 'masterpiece': 1.0, 'hopeless': 1.0, 'clumsy': 1.0, 'conviction': 0.9846153846153847, 'realistic': 1.0, 'portrayal': 1.0, 'reveals': 1.0, 'uneven': 0.9880952380952381, 'unpredictable': 1.0, 'epic': 1.0, 'considerable': 1.0, 'enables': 1.0, 'flashes': 1.0, 'vicious': 1.0, 'jagged': 1.0, 'slick': 1.0, 'essentially': 1.0, 'haunting': 1.0, 'pale': 1.0, 'affecting': 1.0, 'intelligent': 0.9914893617021276, 'backward': 1.0, 'thrill': 1.0, 'powerful': 1.0, 'torture': 1.0, 'celebrates': 1.0, 'strongest': 1.0, 'irrelevant': 1.0, 'canvas': 1.0, 'smug': 1.0, 'captures': 1.0, 'offensive': 1.0, 'tale': 0.9943342776203966, 'sitcom': 1.0, 'abuse': 1.0, 'suffers': 1.0

In [104]:
acc_diff = dict()
for k in ori_acc:
    acc_diff[k] = abs(ori_acc[k] - generated_acc[k])
    

for idx, (k, v) in enumerate(acc_diff.items()):
    if v < 0.3:
        continue
    print("{}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(k, v, ori_acc[k], 
                                                  generated_acc[k], ",".join(syns[k][:3])))

oblivious	0.32	1.00	0.68	insensitive,uncaring,unsympathetic
recycled	0.89	1.00	0.11	recycle,recycling,recycles
cruel	0.31	1.00	0.69	brutal,brutish,ruthless
seal	0.40	1.00	0.60	sealed,hermetic,stamped
casts	0.96	1.00	0.04	castings,throws,puts
slick	0.77	1.00	0.23	blot,blemish,spot
pale	0.45	1.00	0.55	bali,faint,livid
sitcom	0.43	1.00	0.57	comic,sitcoms,comedian
juvenile	0.81	1.00	0.19	juveniles,underage,minors
affection	0.41	1.00	0.59	tenderness,fondness,ailment
capable	0.60	1.00	0.40	able,ability,capacity
deadly	0.89	1.00	0.11	mortal,fatal,lethal
dramatically	0.87	1.00	0.13	drastically,markedly,significantly
longest	0.84	1.00	0.16	tallest,longer,most
fallen	0.58	1.00	0.42	declined,fell,shrunk
qualities	0.32	1.00	0.68	qualifications,qualification,attributes
glossy	0.68	0.95	0.28	shiny,lustrous,gloss
rises	0.49	0.97	0.48	rise,soars,soar
graceful	0.45	1.00	0.55	elegant,tasteful,courtesy
problematic	0.44	0.97	0.53	difficult,tricky,challenging
harmless	0.65	1.00	0.35	inoffensive,innocuous,b